<a href="https://colab.research.google.com/github/zzunebye/Capstone-code-data/blob/main/_RumorEval_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

/content
Mounted at ./MyDrive


In [7]:
cd MyDrive/MyDrive/Capstone/code_data

[Errno 2] No such file or directory: 'MyDrive/MyDrive/Capstone/code_data'
/content/MyDrive/MyDrive/Capstone/code_data


In [1]:
import pandas as pd
import numpy as np

from glob2 import glob
import json

In [2]:
path = "../pheme-rnr-dataset"
events = ['charliehebdo', 'ferguson',
          'germanwings-crash', 'ottawashooting', 'sydneysiege']
jsons = []

In [3]:
for i, event in enumerate(events):
    # print(i, event)
    jsons.append(glob('%s/%s/*/*/source-tweet/*.json' % (path, event)))


In [4]:
len(jsons)

5

In [5]:
targets = []
features = []
for index, dataset in enumerate(jsons):
    targetEvent = []
    dataEvent = []
    count = 0  # help var
    for jsonFile in dataset:
        count += 1
        if jsonFile.find("non-rumours") == -1:
            targetEvent.append(1)
        else:
            targetEvent.append(0)

        with open(jsonFile, 'r') as f:
            for l in f.readlines():
                if not l.strip():  # skip empty lines
                    continue
                json_data = json.loads(l)
                # print (json_data,"\n\n")
                dataEvent.append(json_data)
    print(index, events[index], len(targetEvent), len(dataEvent))
    targets.append(targetEvent)
    features.append(dataEvent)


0 charliehebdo 2079 2079
1 ferguson 1143 1143
2 germanwings-crash 469 469
3 ottawashooting 890 890
4 sydneysiege 1221 1221


In [9]:
len(targets)
len(targets[0])

2079

In [13]:
# targets은 targetEvent들을 리스트에 담은 것
target_list = []
for event in targets:
    for elem in event:
        target_list.append(elem)
target = pd.DataFrame(target_list, columns=["target"])

## Iterate through each tweet => output the dict of selected twt obects


In [30]:
extracted_features = []

extracted = []
# total_tokens_l = []

for obj_list in features:
    extracted_event = []
    for obj in obj_list:
        output_f = dict()
        # print(tweet_obj['text'])
        output_f['text'] = obj['text']
        # urls_dicts = obj['entities']['urls']
        # output_f['verified'] = obj['user']['verified']
        extracted_event.append(output_f)
    extracted_features.append(extracted_event)
# unk_tokens_l = list(set(total_tokens_l))
# print("Number of total tokens appeared: {}\nNumber of unique tokens appeared: {}\n".format(
# len(total_tokens_l), len(unk_tokens_l)))  # number of tokens and unique tokens


## extracted_features를 temp로 변환하고 events 피쳐를 추가

In [31]:
extracted_df = []
for i, data in enumerate(extracted_features):
    temp = pd.DataFrame(data)
    temp["Event"] = events[i]
    extracted_df.append(pd.DataFrame(temp))

In [33]:
final = pd.concat(extracted_df, ignore_index=True)
final = pd.concat([final, target], axis=1)
final.head()

,text,Event,target
0,BREAKING: Armed man takes hostage in kosher gr...,charliehebdo,1
1,"#CharlieHebdo killers dead, confirmed by genda...",charliehebdo,1
2,"Top French cartoonists Charb, Cabu, Wolinski, ...",charliehebdo,1
3,Police have surrounded the area where the #Cha...,charliehebdo,1
4,PHOTO: Armed gunmen face police officers near ...,charliehebdo,1


In [34]:
print(final.shape)
final.sample(5)

(5802, 3)


,text,Event,target
3877,Police say there are 3 shooting scenes: Parlia...,ottawashooting,1
4877,"""The situation in sydney must be hard for 5sos...",sydneysiege,1
4435,Shot from inside caucus room after shooting @c...,ottawashooting,0
3112,Harvard Law students stand in solidarity w/#Fe...,ferguson,0
214,Gallery: Tank &amp; armoured car seen near #Ch...,charliehebdo,1


In [38]:
final.to_csv('./data/_PHEME_text.csv', index = False)
target.to_csv('./data/_PHEME_target.csv', index = False)
# d = pd.read_csv("./data/_PHEME_text.csv")